In [1]:
import ingestor, modeller, fitter
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

In [3]:
from importlib import reload
reload(ingestor)
reload(modeller)
reload(fitter)

<module 'fitter' from '/media/dwu402/Data/wrap-mad/fitter.py'>

In [4]:
plt.style.use('seaborn-notebook')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from cycler import cycler

new_color = cycler(color=["k"])
plt.rcParams['axes.prop_cycle'] = plt.rcParams['axes.prop_cycle'].concat(new_color)

In [5]:
context = ingestor.Context("runs/augmented_all.run")

In [6]:
def replace(l, idx, val):
    return l[:idx] + [val] + l[idx+1:]

In [7]:
solvers = []

In [8]:
Nalpha = 49
Nrho = 49
alphas = np.logspace(-7, 5, num=Nalpha)
rhos = np.logspace(-6, 6, num=Nrho)

In [ ]:
for alpha in alphas:
    print(f"alpha : {alpha}")
    context.update('fitting_configuration', 
                   {'regularisation_parameter': replace(context.fitting_configuration['regularisation_parameter'], 
                                                        2, alpha)
                   })
    solver = fitter.Fitter(context)
    solvers.append(solver)
    for rho in rhos:
        solver.solve(rho)

alpha : 1e-07


## Visualisations

In [ ]:
funs = [list(map(lambda x: x[0]['fun'], solver.solutions.values())) for solver in solvers]
sols = [solver.solutions for solver in solvers]

rs = []
fs = []
ds = []

for alpha, solver, sol, fun in zip(alphas, solvers, sols, funs):
    rs.append([solver.regularisation(sol[str(rho)][0].x)/alpha  for rho in rhos])
    fs.append([fun[idx] for idx, _ in enumerate(rhos)])
    ds.append([(solver.problems[0].cache.results[fitter.tokey(rho, sol[str(rho)][0].x)].fun - fs[-1][idx])/rho 
               for idx, rho in enumerate(rhos)])


In [ ]:
fs, rs, ds = (np.array(fs), np.array(rs), np.array(ds))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.contourf(np.log10(fs), np.log10(rs), np.log10(abs(ds)), levels=50)
ax.plot_wireframe(np.log10(fs), np.log10(rs), np.log10(abs(ds)), linewidth=0.25)
# ax.scatter(np.log10(fs), np.log10(rs), np.log10(abs(ds)), s=5)
ax.set_xlabel('Data Fit')
ax.set_ylabel('Outer Regularisation Term')
ax.set_zlabel('Gradient Field')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(*np.log10(np.meshgrid(rhos,alphas)), np.log(fs))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel("H")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(*np.log10(np.meshgrid(rhos,alphas)), np.log10(rs))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel("L")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(*np.log10(np.meshgrid(rhos,alphas)), np.log(np.abs(ds)))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel("D")

In [ ]:
from scipy import interpolate

rhomesh, alphamesh= np.log10(np.meshgrid(rhos,alphas))

In [ ]:
fs_smooth = interpolate.RectBivariateSpline(np.log10(alphas), np.log10(rhos), np.log10(fs), kx=4, ky=4, s=40)
rs_smooth = interpolate.RectBivariateSpline(np.log10(alphas), np.log10(rhos), np.log10(rs), kx=4, ky=4, s=600)
ds_smooth = interpolate.RectBivariateSpline(np.log10(alphas), np.log10(rhos), np.log10(np.abs(ds)), kx=4, ky=4, s=600)

In [ ]:
fs_eval = np.array([fs_smooth(a, r).item() for r,a in zip(rhomesh.flatten(), alphamesh.flatten())]).reshape(Nalpha, Nrho)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(rhomesh, alphamesh, fs_eval)
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel(r"$H$")

In [ ]:
rs_eval = np.array([rs_smooth(a, r).item() for r,a in zip(rhomesh.flatten(), alphamesh.flatten())]).reshape(Nalpha, Nrho)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(rhomesh, alphamesh, rs_eval)
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel(r"$L$")

In [ ]:
ds_eval = np.array([ds_smooth(a, r).item() for r,a in zip(rhomesh.flatten(), alphamesh.flatten())]).reshape(Nalpha, Nrho)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(rhomesh, alphamesh, ds_eval)
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel(r"$D$")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(fs_eval, rs_eval, ds_eval, cmap='cividis')
ax.plot_wireframe(fs_eval, rs_eval, ds_eval, linewidth=0.25)
ax.set_xlabel('H')
ax.set_ylabel('L')
ax.set_zlabel('D')

In [ ]:
x_u = lambda r, a: np.array([f(a, r, dx=1).item() for f in [fs_smooth, rs_smooth, ds_smooth]])
x_v = lambda r, a: np.array([f(a, r, dy=1).item() for f in [fs_smooth, rs_smooth, ds_smooth]])
x_uu = lambda r, a: np.array([f(a, r, dx=2).item() for f in [fs_smooth, rs_smooth, ds_smooth]])
x_uv = lambda r, a: np.array([f(a, r, dx=1, dy=2).item() for f in [fs_smooth, rs_smooth, ds_smooth]])
x_vv = lambda r, a: np.array([f(a, r, dy=2).item() for f in [fs_smooth, rs_smooth, ds_smooth]])
n = lambda r, a: np.cross(x_u(r,a), x_v(r,a))/np.linalg.norm(np.cross(x_u(r,a), x_v(r,a)))

fundamental_1 = lambda r, a: np.array([[np.dot(x_u(r,a), x_u(r,a)), np.dot(x_u(r,a), x_v(r,a))],
                                       [np.dot(x_u(r,a), x_v(r,a)), np.dot(x_v(r,a), x_v(r,a))]])
fundamental_2 = lambda r, a: np.array([[np.dot(x_uu(r,a), n(r,a)), np.dot(x_uv(r,a), n(r,a))],
                                       [np.dot(x_uv(r,a), n(r,a)), np.dot(x_vv(r,a), n(r,a))]])

curvature = lambda r, a: np.linalg.eigvals(np.linalg.solve(fundamental_1(r,a), fundamental_2(r,a)))
gaussian_curvature = lambda r, a: np.prod(curvature(r,a))
mean_curvature = lambda r, a: np.mean(curvature(r, a))

In [ ]:
gcurvs = np.array([gaussian_curvature(r, a) for r, a in zip(rhomesh.flatten(), alphamesh.flatten())]).reshape(Nalpha, Nrho)
mcurvs = np.array([mean_curvature(r, a) for r, a in zip(rhomesh.flatten(), alphamesh.flatten())]).reshape(Nalpha, Nrho)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(rhomesh, alphamesh, np.log10(gcurvs))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel(r"$\kappa$")
ax.set_title("Gaussian Curvature")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(rhomesh, alphamesh, np.log10(mcurvs))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel(r"$\kappa$")
ax.set_title("Mean Curvature")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.imshow(np.log10(gcurvs).T + np.log10(mcurvs).T)
ax.set_xticks(range(0, Nalpha, 4))
ax.set_xticklabels(np.log10(alphas[::4]))
ax.set_yticks(range(0, Nrho, 4))
ax.set_yticklabels(np.round(np.log10(rhos[::4]), 2))
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\rho$")
fig.colorbar(im)

In [ ]:
def ctfn(mk, gk):
    tol = 1e-6
    if mk > tol:
        if gk > tol:
            return 1
        elif gk < -tol:
            return 0.5
    elif mk < -tol:
        if gk > tol:
            return -0.5
        elif gk <-tol:
            return -1
    return 0

curv_type = np.array([ctfn(M,G) for M, G in zip(mcurvs.flatten(), gcurvs.flatten())]).reshape(mcurvs.shape)

fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.imshow(curv_type.T, cmap='Spectral')
ax.set_xticks(range(0, Nalpha, 4))
ax.set_xticklabels(np.log10(alphas[::4]))
ax.set_yticks(range(0, Nrho, 4))
ax.set_yticklabels(np.round(np.log10(rhos[::4]), 2))
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\rho$")
fig.colorbar(im)

In [ ]:
fs

In [ ]:
rs

In [ ]:
ds

In [ ]:
i = np.argmax(mcurvs)
print(10**rhomesh.flatten()[i])
print(10**alphamesh.flatten()[i])

In [ ]:
# norm calcs
error_norms = []
for f, d, r in zip(fs, ds, rs):
    error_norms.append([np.linalg.norm([fi,di,ri]) for fi,di,ri in zip(f,d,r)])
    
error_norms = np.array(error_norms)
    
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(*np.log(np.meshgrid(rhos,alphas)), np.log(error_norms))

In [ ]:
# Perform a profiling step
ideal_rhos = []
jdxs = []
for alpha, en in zip(alphas, error_norms):
    jdxs.append(np.argmin(en))
    ideal_rhos.append((rhos[np.argmin(en)], min(en)))
ideal_rhos = np.array(ideal_rhos)
plt.figure()
plt.loglog(alphas, ideal_rhos[:,0])
plt.title(r"$\rho$ for different $\alpha$")
plt.xlabel("$\\alpha$")
plt.ylabel("Ideal $\\rho$")

In [ ]:
# Transpose profiling

ideal_alphas = []
kdxs = []
for rho, en in zip(rhos, error_norms.T):
    kdxs.append(np.argmin(en))
    ideal_alphas.append((alphas[np.argmin(en)], min(en)))
ideal_alphas = np.array(ideal_alphas)
plt.figure()
plt.loglog(rhos, ideal_alphas[:,0])
plt.title(r"$\alpha$ for different $\rho$")
plt.xlabel("$\\rho$")
plt.ylabel("Ideal $\\alpha$")

In [ ]:
# construct contraction mapping plot
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot(*np.log10((rhos, ideal_alphas[:,0], ideal_alphas[:,1])))
ax.plot(*np.log10((ideal_rhos[:,0], alphas, ideal_rhos[:,1])))
ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$\alpha$")
ax.set_zlabel("Norm")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(fs_eval, rs_eval, ds_eval, cmap='cividis', alpha=0.75)
# ax.plot_wireframe(fs_eval, rs_eval, ds_eval, linewidth=0.25)

# plot in points
ax.plot(fs_eval[range(Nalpha), jdxs], rs_eval[range(Nalpha), jdxs], ds_eval[range(Nalpha), jdxs], 'ko-')
ax.plot(fs_eval[kdxs, range(Nrho)], rs_eval[kdxs, range(Nrho)], ds_eval[kdxs, range(Nrho)], 'ro-')

ax.set_xlabel('H')
ax.set_ylabel('L')
ax.set_zlabel('D')